I will be working with fault code data and vehicle onboard diagnostic data to try and predict an upcoming full derate. These are indicated by an SPN 5246.

In [1]:
import pandas as pd
from datetime import datetime
import geopandas as gpd
from geopy.distance import distance
import numpy as np
from sklearn.impute import SimpleImputer

In [2]:
parse_dates=['EventTimeStamp']

In [3]:
faults = pd.read_csv('data/J1939Faults.csv', low_memory=False, parse_dates=['EventTimeStamp'])
service_fault = pd.read_excel('data/ServiceFaultCodes.xlsx')
vehicle_Diagnostics = pd.read_csv('data/VehicleDiagnosticOnboardData.csv')

/Users/alisoncordoba/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [4]:
vehicle_Diagnostics = pd.DataFrame(vehicle_Diagnostics)

# **Cleaning Faults data**

In [5]:
# filter out rows that have more than 5 characters in the 'EquipmentID' column
faults =faults[faults['EquipmentID'].str.len() <= 5]

In [6]:
faults = faults.drop(columns = ["actionDescription", "faultValue"])

In [7]:
faults = faults[(faults['Latitude'] != '36.0666667') & (faults['Longitude'] != '-86.4347222')]
faults = faults[(faults['Latitude'] != '35.5883333') & (faults['Longitude'] != '-86.4438888')]
faults = faults[(faults['Latitude'] != '36.1950') & (faults['Longitude'] != '-83.174722')]

- Remove faults occurring in the vicinity of the service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722)

In [8]:
for lat, lon in [(36.0666667, -86.4347222), (35.5883333, -86.4438888), (36.1950, -83.174722)]:
    
    faults = faults.loc[~((abs(lat - faults['Latitude']) <= 0.01) &
                          (abs(lon - faults['Longitude']) <= 0.01))]

- selected unique trucks with partial derate and complete derate (I can change the name, currently I went with ‘total’ derate) and then I compared them.

In [9]:
all_trucks = faults['EquipmentID'].unique()
partial_derate = faults.loc[(faults['spn'] == 1569) & (faults['fmi'] == 31)]['EquipmentID'].unique()
total_derate = faults.loc[faults['spn'] == 5246]['EquipmentID'].unique()

partial_derate_only = partial_derate[np.isin(partial_derate, total_derate, invert=True)]
total_derate_only = total_derate[np.isin(total_derate, partial_derate, invert=True)]
partial_and_total_derate = np.intersect1d(partial_derate, total_derate)
no_derate = all_trucks[np.isin(all_trucks, partial_derate_only, invert=True) | np.isin(all_trucks, total_derate_only, invert=True)]

- Dropping dates after 2011 

In [10]:
faults = faults.loc[faults['EventTimeStamp'].dt.year > 2011]

In [11]:
faults = faults.loc[faults['active'] != False]

- To filter out the events near the service stations:

In [12]:
faults.loc[faults['spn'] == 5246]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
2089,2090,1011009,2015-02-23 05:05:44,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23.000
2971,2972,1026305,2015-02-23 15:54:22,NaN,unknown,unknown,unknown,unknown,0,5246,0,True,1,1487,105369355,28.077361,-81.897083,2015-02-23 15:54:58.000
5713,5714,1070646,2015-02-25 13:53:08,NaN,unknown,unknown,unknown,unknown,0,5246,0,True,1,1329,105400037,39.399583,-82.974768,2015-02-25 13:56:31.000
6534,6535,1097942,2015-02-26 22:24:29,NaN,04993120*00021657*082113134117*07700053*I0*BBZ*,79466573,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1419,105355995,37.596805,-85.865555,2015-02-26 22:25:05.000
6628,6629,1104361,2015-02-27 09:09:56,NaN,04993120*00054166*082113134117*07700053*I0*BBZ*,79487658,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1486,105338831,40.534259,-76.431805,2015-02-27 09:10:33.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1173036,1232200,116794344,2020-01-13 13:18:31,NaN,NaN,NaN,NaN,NaN,49,5246,19,True,46,302,105418777,38.192824,-85.859490,2020-01-13 13:19:06.000
1178571,1238712,119571469,2020-02-03 15:46:46,NaN,04384413*22246857*121817205924*60701721*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,True,1,2211,105329862,35.833935,-86.410925,2020-02-03 15:47:23.000
1179414,1239555,119959276,2020-02-06 07:45:08,NaN,04358814*06005963*051718174436*09401683*G1*BDR*,79897320,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1854,105385876,35.943611,-83.823009,2020-02-06 07:45:44.000
1181700,1241841,120905759,2020-02-13 13:32:39,NaN,04358814*06026985*051718174436*09401683*G1*BDR*,79903054,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1872,105301976,35.707268,-81.397037,2020-02-13 13:33:15.000


In [13]:
#Create new column EventTimeStamp_DateOnly with only date part
faults['EventTimeStamp_DateOnly']= pd.to_datetime(faults['EventTimeStamp'], format='%Y-%m-%d')
faults['LocationTimeStamp_DateOnly']= pd.to_datetime(faults['LocationTimeStamp'],  format='%Y-%m-%d')

faults['EventTimeStamp_DateOnly'] = pd.to_datetime(faults['EventTimeStamp_DateOnly'])
faults['LocationTimeStamp_DateOnly'] = pd.to_datetime(faults['LocationTimeStamp_DateOnly'])

In [14]:
#Filter out 'R1762', 'R1764'
faults = faults[~faults[['EquipmentID']].isin(['R1762', 'R1764', '2185A']).any(axis=1)]

In [15]:
faults = faults.astype({'EquipmentID': 'int'})

- Basic EDA to check how the data looks like:

In [16]:
faults.shape
#service_fault.shape
#vehicle_Diagnostics.shape

(546664, 20)

In [17]:
faults.isna().sum()
#service_fault.isna().sum()
#vehicle_Diagnostics.isna().sum()

RecordID                           0
ESS_Id                             0
EventTimeStamp                     0
eventDescription               29659
ecuSoftwareVersion            132422
ecuSerialNumber               153494
ecuModel                       28103
ecuMake                        28103
ecuSource                          0
spn                                0
fmi                                0
active                             0
activeTransitionCount              0
EquipmentID                        0
MCTNumber                          0
Latitude                           0
Longitude                          0
LocationTimeStamp                  0
EventTimeStamp_DateOnly            0
LocationTimeStamp_DateOnly         0
dtype: int64

- Creating a copy of Faults dataframe to be able to make changes without affecting the original data. 

In [18]:
faults_copy = faults 
Diagnostics = vehicle_Diagnostics

# **Find vehicle EquipmentID for which the derate has happened**

**This code was shared by AJAY**

In [19]:
#df_faults['EventTimeStamp_DateOnly'] = pd.to_datetime(df_faults['EventTimeStamp_DateOnly'])
df_faults= (
    faults_copy.groupby(['EquipmentID','EventTimeStamp','RecordID' ,'EventTimeStamp_DateOnly','active', 'spn', 'fmi'])
    .size().reset_index(name='count')
    .sort_values(by=['EquipmentID','EventTimeStamp','RecordID' ], ascending = [False, False, False])
)
df_faults.head(30)

,EquipmentID,EventTimeStamp,RecordID,EventTimeStamp_DateOnly,active,spn,fmi,count
546663,2381,2020-03-06 11:56:38,1248422,2020-03-06 11:56:38,True,37,18,1
546662,2380,2020-03-04 10:06:10,1247712,2020-03-04 10:06:10,True,1761,17,1
546661,2377,2020-03-06 14:14:13,1248457,2020-03-06 14:14:13,True,111,18,1
546660,2377,2020-03-03 07:47:01,1247361,2020-03-03 07:47:01,True,111,18,1
546659,2377,2020-02-28 04:56:18,1246549,2020-02-28 04:56:18,True,111,18,1
546658,2377,2020-02-12 05:42:02,1241225,2020-02-12 05:42:02,True,111,18,1
546657,2377,2020-01-14 13:46:39,1232638,2020-01-14 13:46:39,True,1231,8,1
546656,2377,2020-01-14 13:42:40,1232624,2020-01-14 13:42:40,True,1231,8,1
546655,2377,2020-01-14 13:42:40,1232623,2020-01-14 13:42:40,True,84,23,1
546654,2377,2020-01-14 13:42:34,1232622,2020-01-14 13:42:34,True,84,2,1


In [20]:
df_faults['DummyDerate_5246'] = np.where(df_faults['spn'] == 5246, 1, 0)
df_faults['DummyDerate_1569'] = np.where(df_faults['spn'] == 1569, 1, 0)

In [21]:
df_faults_5246= (
    df_faults.groupby(['EquipmentID','EventTimeStamp','RecordID' ,'EventTimeStamp_DateOnly','active', 'spn', 'fmi', 'DummyDerate_5246'])
    .size().reset_index(name='count')
    .sort_values(by=['EquipmentID','EventTimeStamp','DummyDerate_5246','RecordID' ], ascending = [False, False, False, False])
)

df_faults_5246.head

#eyeball the dataframe and save it. 
#df_faults_5246.to_csv('data/df_faults_5246.csv')  

<bound method NDFrame.head of         EquipmentID      EventTimeStamp  RecordID EventTimeStamp_DateOnly  \
546663         2381 2020-03-06 11:56:38   1248422     2020-03-06 11:56:38   
546662         2380 2020-03-04 10:06:10   1247712     2020-03-04 10:06:10   
546661         2377 2020-03-06 14:14:13   1248457     2020-03-06 14:14:13   
546660         2377 2020-03-03 07:47:01   1247361     2020-03-03 07:47:01   
546659         2377 2020-02-28 04:56:18   1246549     2020-02-28 04:56:18   
...             ...                 ...       ...                     ...   
4               301 2015-05-28 13:31:41     68953     2015-05-28 13:31:41   
3               301 2015-05-21 13:57:35     61706     2015-05-21 13:57:35   
2               301 2015-05-18 09:34:05     57330     2015-05-18 09:34:05   
1               301 2015-05-13 08:22:32     51363     2015-05-13 08:22:32   
0               301 2015-05-11 13:11:20     49415     2015-05-11 13:11:20   

        active   spn  fmi  DummyDerate_5246  

In [22]:
from datetime import datetime, timedelta

In [23]:
df_faults_1569= (
    df_faults.groupby(['EquipmentID','EventTimeStamp','RecordID' ,'EventTimeStamp_DateOnly','active', 'spn', 'fmi', 'DummyDerate_1569'])
    .size().reset_index(name='count')
    .sort_values(by=['EquipmentID','EventTimeStamp','DummyDerate_1569','RecordID' ], ascending = [False, False, False, False])
)

#df_faults_1569
df_faults_1569[(df_faults_1569['spn']  == 1569) ]

#eyeball the dataframe and save it. 
#df_faults_1569.to_csv('data/df_faults_1569.csv')

,EquipmentID,EventTimeStamp,RecordID,EventTimeStamp_DateOnly,active,spn,fmi,DummyDerate_1569,count
537451,2120,2018-11-16 06:14:51,1080283,2018-11-16 06:14:51,True,1569,31,1,1
526224,2027,2018-07-22 19:21:30,1038825,2018-07-22 19:21:30,True,1569,31,1,1
525167,2021,2019-08-30 18:05:29,1192545,2019-08-30 18:05:29,True,1569,31,1,1
525116,2020,2019-01-06 22:25:00,1102562,2019-01-06 22:25:00,True,1569,31,1,1
524976,2019,2017-06-06 09:59:45,801682,2017-06-06 09:59:45,True,1569,31,1,1
...,...,...,...,...,...,...,...,...,...
7411,308,2018-02-07 16:25:53,962200,2018-02-07 16:25:53,True,1569,31,1,1
7409,308,2018-02-07 12:16:54,962038,2018-02-07 12:16:54,True,1569,31,1,1
7408,308,2018-02-07 08:11:54,961824,2018-02-07 08:11:54,True,1569,31,1,1
7092,307,2017-07-17 16:04:34,829656,2017-07-17 16:04:34,True,1569,31,1,1


#When calling this function for spn 5246 or 1569. Update the if condition below.
# The following helps to separate the desired derates
def  GetFilteredSPNbyDays(df_faults, windowTimeframeUnit, day_window):
    df_new = pd.DataFrame(columns = ['RecordID','EquipmentID', 'EventTimeStamp','EventTimeStamp_DateOnly','active', 'spn', 'fmi','Derate', 'group','target'])  
    #print(df_new)
    #df_new = df_new.astype({'EquipmentID': 'int'})
    dts_evt = ""
    dts_evt_max=""

    hasDerate = False
    counter =0
    derateGroupCounter = 0
    # loop through rows of original dataframe and assign new values to columns of new dataframe
    for index, row in df_faults.iterrows():
        counter+=1       
        #print('derateGroupCounter== ' + str(derateGroupCounter))
        #if((row['spn'] == 1569) & (row['fmi'] == 31)): 
        #print('-1 - . EquipmentID = ' + str(row['EquipmentID']))
        #if((row['spn'] == 5246)): 
        if((row['spn'] == 1569) & (row['fmi'] == 31)): 
            #print('0. spn Check Counter = ' + str(counter))          
            #print('0 - A. Counter = ' + str(hasDerate))
            derateGroupCounter +=1
            
            #Ak Commented if(hasDerate == False):
            
            hasDerate = True
            df_new.loc[index, 'Derate'] = pd.to_numeric(1)
            df_new.loc[index, 'target'] = 0
            dts_evt = row['EventTimeStamp'] 
            #dts_evt_max = dts_evt - timedelta(days=day_window)
            if(windowTimeframeUnit == "hours"):                   
                dts_evt_max = dts_evt - timedelta(hours=day_window)
                #print("hours = dts_evt_max= " + str(dts_evt_max))
            elif(windowTimeframeUnit == "days"):                   
                dts_evt_max = dts_evt - timedelta(days=day_window)
                #print("days = dts_evt_max= " + str(dts_evt_max))
            #add cols
            #print('1. Counter = ' + str(counter))
            #print('1. RecordID = ' + str(row['RecordID']))
            #print('dts_evt =' + str(dts_evt))
            #print('dts_evt_max =' + str(dts_evt_max))
           
            df_new.loc[index, 'RecordID'] = row['RecordID']
            df_new.loc[index, 'EquipmentID'] = pd.to_numeric(row['EquipmentID']) 
            df_new.loc[index, 'EventTimeStamp'] = row['EventTimeStamp']
            df_new.loc[index, 'EventTimeStamp_DateOnly'] = row['EventTimeStamp_DateOnly'] 
            df_new.loc[index, 'active'] = row['active']
            df_new.loc[index, 'spn'] = row['spn']
            df_new.loc[index, 'fmi'] = row['fmi']
            df_new.loc[index, 'group'] = derateGroupCounter
        else:
            #print('18. Counter = ' + str(counter))
            if(dts_evt_max != ""):
                #print('20. Counter = ' + str(counter))
                #print('21. RecordID = ' + str(row['RecordID']))
                #print('22 In Else after dts_evt_max != ' + str((row['EventTimeStamp'] > dts_evt_max)))
                #print((row['EventTimeStamp'] > dts_evt_max))
                if((row['EventTimeStamp'] > dts_evt_max) & (hasDerate)) :
                    #print('EventTimeStamp > dts_evt_max' + str(dts_evt_max) + " ---- " + str(row['EventTimeStamp_DateOnly']))
                    #print('23 EventTimeStamp > dts_evt_max' + str(dts_evt_max) + " ---- " + str(row['EventTimeStamp_DateOnly']))
                    df_new.loc[index, 'Derate'] = pd.to_numeric(0)
                    df_new.loc[index, 'target'] = 1
                    df_new.loc[index, 'RecordID'] = row['RecordID']
                    df_new.loc[index, 'EquipmentID'] = pd.to_numeric(row['EquipmentID']) 
                    df_new.loc[index, 'EventTimeStamp'] = row['EventTimeStamp']
                    df_new.loc[index, 'EventTimeStamp_DateOnly'] = row['EventTimeStamp_DateOnly'] 
                    df_new.loc[index, 'active'] = row['active']
                    df_new.loc[index, 'spn'] = row['spn']
                    df_new.loc[index, 'fmi'] = row['fmi']
                    df_new.loc[index, 'group'] = derateGroupCounter
                    #print('3. Counter = ' + str(counter))
                else:
                    #print('ELSE  ' + str(dts_evt_max)+ " ---- " +  str(row['EventTimeStamp_DateOnly']))
                    hasDerate = False
                    dts_evt = ""
                    dts_evt_max = ""
                    #print('3. Counter else else = ' + str(counter))
    #print(df_new) 
    
    return df_new

#get the filtered SPN  5246 for 6 hours timeframe
df_filtered_5246 = pd.DataFrame()
df_filtered_5246.info()
chunkcount = 0
for chunk in pd.read_csv("data/df_faults_5246.csv", chunksize=20000, parse_dates=['EventTimeStamp']):
    chunkcount +=1
    dfW = GetFilteredSPNbyDays(chunk,"hours", 6)
    print(chunkcount)
    df_filtered_5246 = pd.concat([df_filtered_5246, dfW])
    
#save the dataframe to compare with the one in prior step to visually compare few rows. This csv contains the target variable
df_filtered_5246.to_csv('data/df_filtered_5246.csv') 

#get the filtered SPN  1569 for 6 hours timeframe
df_filtered_1569 = pd.DataFrame()
df_filtered_1569.info()
chunkcount = 0
for chunk in pd.read_csv("data/df_faults_1569.csv", chunksize=20000, parse_dates=['EventTimeStamp']):
    chunkcount +=1
    dfW = GetFilteredSPNbyDays(chunk,"hours", 6)
    print(chunkcount)
    df_filtered_1569 = pd.concat([df_filtered_1569, dfW])
    
df_filtered_1569
#save the dataframe to compare with the one in prior step 
df_filtered_1569.to_csv('data/df_filtered_1569.csv')

Changing the shape of Diagnostics to be able to merge it. 

In [24]:
# Create a wider table from long table, and drop the Name
Diagnostics = Diagnostics.pivot(index="FaultId", columns="Name", values="Value").reset_index()
Diagnostics

Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1187331,1248455,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,...,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,...,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
1187333,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0,27.26,221.7312,...,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [25]:
Diagnostics.isna().sum()

Name
FaultId                            0
AcceleratorPedal              655446
BarometricPressure            601359
CruiseControlActive           612419
CruiseControlSetSpeed         610877
DistanceLtd                   601516
EngineCoolantTemperature      601264
EngineLoad                    601714
EngineOilPressure             601091
EngineOilTemperature          603423
EngineRpm                     600414
EngineTimeLtd                 605969
FuelLevel                     684540
FuelLtd                       602140
FuelRate                      602098
FuelTemperature               888225
IgnStatus                     578881
IntakeManifoldTemperature     601044
LampStatus                         0
ParkingBrake                  787363
ServiceDistance              1187120
Speed                         603419
SwitchedBatteryVoltage       1073276
Throttle                      766832
TurboBoostPressure            603984
dtype: int64

In [26]:
#Diagnostics['CruiseControlActive'].astype('bool')
#Diagnostics['IgnStatus'].astype('bool')


#Diagnostics['CruiseControlActive'] = Diagnostics.CruiseControlActive.astype(bool)
#Diagnostics['IgnStatus'] = Diagnostics.IgnStatus.astype(bool)



- Finding and dropping commas in columns using the following code:

In [27]:
# Remove commas from all 11 columns
for col in Diagnostics.columns[:21]:
    Diagnostics[col] = Diagnostics[col].astype(str).str.replace(',', '')

# Convert all columns to numeric
Diagnostics = Diagnostics.apply(pd.to_numeric, errors='coerce')

Diagnostics

Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0.0,14.2100,NaN,66.48672,423178.70000,100.4,11.0,0.00,96.74375,...,NaN,NaN,78.8,1023,NaN,NaN,0.00000,3276.75,NaN,0.00
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1187331,1248455,100.0,14.5000,NaN,64.62260,423937.90000,185.0,51.0,37.12,211.49370,...,32.0,NaN,98.6,18431,NaN,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,0.0,14.3550,NaN,66.48672,465925.40000,186.8,62.0,41.18,212.84380,...,NaN,NaN,91.4,17407,NaN,NaN,66.57410,NaN,100.0,6.96
1187333,1248457,1.6,14.4275,NaN,67.72946,28606.65625,181.4,0.0,27.26,221.73120,...,NaN,NaN,100.4,1023,NaN,NaN,11.84489,14.10,100.0,1.74


In [28]:
Diagnostics = Diagnostics.drop(columns=['CruiseControlActive', 'IgnStatus', 'ParkingBrake'])

The following code will allow the transformation of the nan values to the mean per truck in the columns. 

In [29]:
#imputer =  SimpleImputer(strategy='median', fill_value=None)

#Impute the missing values using the median strategy
#imputed_Diagnostics = imputer.fit_transform(Diagnostics)


#imputed_Diagnostics

In [30]:
#[x for x in Diagnostics.columns if x not in
#    imputer.get_feature_names_out()]

In [31]:
# Convert the imputed array back to dataframe
#copy_imputer = pd.DataFrame(imputed_Diagnostics, columns=Diagnostics.columns)

# Replace NaN values with the median value
#copy_imputer

In [32]:
#Merge faults and vehicle dignostic tables
#faults_copy = pd.merge(faults_copy, copy_imputer, left_on='RecordID', right_on='FaultId')
#faults_copy

Repeating the process to check for better results. 

In [33]:
#The following is a copy to avoid data contamination.
Diagnostics_improved = Diagnostics

In [34]:
#Merge faults and vehicle dignostic tables
faults_improved = faults_copy.merge(Diagnostics_improved, left_on='RecordID', right_on='FaultId')
faults_improved

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,...,12300.907429,0.000000,NaN,78.8,1023,NaN,0.000000,3276.75,NaN,0.00
1,2,990360,2015-02-21 11:34:34,NaN,unknown,unknown,unknown,unknown,11,629,...,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN
2,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,...,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN
3,6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,...,70349.809964,4.583399,NaN,111.2,1023,NaN,13.602200,3276.75,NaN,6.67
4,7,990439,2015-02-21 11:40:52,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,...,40961.065437,14.291750,NaN,78.8,1023,NaN,41.534780,3276.75,NaN,20.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546659,1248448,123899434,2020-03-06 13:12:43,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,...,51466.131257,0.620806,NaN,120.2,1279,NaN,0.941766,NaN,100.0,1.16
546660,1248452,123901805,2020-03-06 13:42:48,Low (Severity Medium) Engine Coolant Level,04358814*06030918*051718174436*09401683*G1*BDR*,79904453,6X1u13D1500000000,CMMNS,0,111,...,64491.926797,0.515137,NaN,104.0,2047,NaN,5.932153,NaN,100.0,0.58
546661,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,...,58979.184416,7.647805,32.0,98.6,18431,NaN,65.010960,NaN,73.2,7.83
546662,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,...,65080.105870,8.995086,NaN,91.4,17407,NaN,66.574100,NaN,100.0,6.96


In [35]:
grouped = faults_improved['EquipmentID'].reset_index(drop=True)

grouped 

0         1439
1         1439
2         1369
3         1417
4         1597
          ... 
546659    1936
546660    1886
546661    1994
546662    1850
546663    2377
Name: EquipmentID, Length: 546664, dtype: int64

In [36]:
# Define the columns to include in the grouping
#cols = ['ESS_Id', 'EventTimeStamp','spn', 'fmi', 
#        'activeTransitionCount','MCTNumber', 'LocationTimeStamp', 
#        'FaultId', 'AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed',
 #       'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 
  #      'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 
   #     'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'LampStatus', 
    #    'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']

# Creating a new DataFrame to store the imputed values
#imputed_values = pd.DataFrame(columns=cols)

# Loop over the rows of the DataFrame
#for idx, row in faults_improved.iterrows():

    # Group the current row by EquipmentID and select the columns
#    grouped = faults_improved.loc[idx, cols].groupby(faults_improved.loc[idx, 'EquipmentID']).reset_index()
    
    # Impute the missing values using the mean of each group
#    group_means = grouped.transform(lambda x: x.mean())
#    imputed_row = grouped.transform(lambda x: x.fillna(x.mean()))
    
    # Append the imputed values for the current row to the imputed values DataFrame
#    imputed_values = group_means.append(imputed_row, ignore_index=True)
    
# Replace the missing values in the original DataFrame with the imputed values
#faults_improved[cols] = imputed_values

# Creating a function to replace NAN values in desired columns with the Mean to tbe able to run a Machine Learning model.

The following code just test a simpler imputer in ONE column. 

In [37]:
column = 'Speed' 

imputer = SimpleImputer(strategy='mean')

equipment_imputed = faults_improved.groupby('EquipmentID')[column].apply(lambda x: imputer.fit_transform(x.values.reshape(-1, 1)))

equipment_imputed

EquipmentID
301     [[4.475814], [64.63717], [63.52549], [4.364162...
302     [[10.817992769867548], [10.817992769867548], [...
303     [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....
304     [[33.27734], [22.0247], [40.91341], [34.5929],...
305     [[18.840214921611718], [18.840214921611718], [...
                              ...                        
2375    [[0.0], [1.019437], [60.3798], [64.90416], [0.0]]
2376    [[67.05955], [67.86539], [35.0298], [24.20435]...
2377    [[158.4472], [158.4472], [158.4472], [158.4472...
2380                                              [[0.0]]
2381                                              [[0.0]]
Name: Speed, Length: 1039, dtype: object

In [38]:
equipment_imputed.apply(lambda x : x.flatten()).explode()

for Id in equipment_imputed.index :

    faults_improved.loc[faults_improved['EquipmentID']== Id, column]= equipment_imputed.loc[Id].flatten()


In [39]:
faults_improved[column].isna().sum()

0

- the following code replaces NAN values in ALL columns. 

In [40]:
cols = ['activeTransitionCount','MCTNumber',
        'AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed',
        'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 
        'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 
        'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'LampStatus', 
        'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']

imputer = SimpleImputer(strategy='mean')


for column in cols:
    
    equipment_fixed = faults_improved.groupby('EquipmentID')[column].apply(lambda x: imputer.fit_transform(x.values.reshape(-1, 1)))

    for Id in equipment_imputed.index :
        
        faults_improved.loc[faults_improved['EquipmentID']== Id, column]= equipment_imputed.loc[Id].flatten()


In [41]:
faults_improved[cols].sort_index()

,activeTransitionCount,MCTNumber,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,...,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354
2,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,...,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260
3,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,...,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200
4,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,...,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546659,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,...,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766
546660,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,...,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153
546661,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,...,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960
546662,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,...,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100


In [56]:
df_improved = pd.DataFrame(faults_improved, columns=cols)
df_improved

,activeTransitionCount,MCTNumber,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,...,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354,25.395354
2,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,...,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260,39.459260
3,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,...,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200,13.602200
4,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,...,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780,41.534780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546659,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,...,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766,0.941766
546660,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,...,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153,5.932153
546661,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,...,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960,65.010960
546662,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,...,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100,66.574100


In [60]:
df_improved.isna().sum()

activeTransitionCount        0
MCTNumber                    0
AcceleratorPedal             0
BarometricPressure           0
CruiseControlSetSpeed        0
DistanceLtd                  0
EngineCoolantTemperature     0
EngineLoad                   0
EngineOilPressure            0
EngineOilTemperature         0
EngineRpm                    0
EngineTimeLtd                0
FuelLevel                    0
FuelLtd                      0
FuelRate                     0
FuelTemperature              0
IntakeManifoldTemperature    0
LampStatus                   0
ServiceDistance              0
Speed                        0
SwitchedBatteryVoltage       0
Throttle                     0
TurboBoostPressure           0
tmp                          0
dtype: int64

# **Merging data for Machine Learning models**

In [57]:
faults_5246 = pd.read_csv('data/df_filtered_5246.csv')
faults_1569 = pd.read_csv('data/df_filtered_1569.csv')

In [61]:
faults_5246['tmp'] = 0
faults_1569['tmp'] = 0
df_improved['tmp'] = 0

Diagnostics_5246 = pd.concat(df_improved, faults_5246, ignore_index=True)
#Diagnostics_1569 = pd.merge(df_improved, faults_1569, on=['tmp'])

#Diagnostics_5246 = df.drop('tmp', axis=1)
#Diagnostics_1569 = df.drop('tmp', axis=1)

/var/folders/bk/4_dns4_x6z3_rrb8824x_td40000gn/T/ipykernel_22770/1847119815.py:5: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  Diagnostics_5246 = pd.concat(df_improved, faults_5246, ignore_index=True)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [50]:
faults_1569

,Unnamed: 0,RecordID,EquipmentID,EventTimeStamp,EventTimeStamp_DateOnly,active,spn,fmi,Derate,group,target,tmp
0,9212,1080283,2120,2018-11-16 06:14:51,2018-11-16 06:14:51,True,1569,31,1,1,0,0
1,9213,1080251,2120,2018-11-16 05:14:51,2018-11-16 05:14:51,True,5394,5,0,1,1,0
2,20439,1038825,2027,2018-07-22 19:21:30,2018-07-22 19:21:30,True,1569,31,1,1,0,0
3,20440,1038822,2027,2018-07-22 18:15:41,2018-07-22 18:15:41,True,3251,2,0,1,1,0
4,21496,1192545,2021,2019-08-30 18:05:29,2019-08-30 18:05:29,True,1569,31,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
13396,539572,829620,307,2017-07-17 15:04:44,2017-07-17 15:04:44,True,5743,9,0,155,1,0
13397,539573,829616,307,2017-07-17 15:00:53,2017-07-17 15:00:53,True,5848,9,0,155,1,0
13398,540035,1053642,306,2018-09-07 06:47:05,2018-09-07 06:47:05,True,1569,31,1,1,0,0
13399,540036,1053622,306,2018-09-07 05:48:43,2018-09-07 05:48:43,True,4094,31,0,1,1,0
